In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl
/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
print("Positive examples {0:.0f}".format(data.groupby('sentiment')['sentiment'].count().positive))
print("Negative examples {0:.0f}".format(data.groupby('sentiment')['sentiment'].count().negative))

Positive examples 25000
Negative examples 25000


In [4]:
from collections import Counter
a = ''.join(data['review'].values)
a= a.lower()
a = list(a.split())

Counter(list(a)).most_common(100)

[('the', 635766),
 ('a', 315257),
 ('and', 313549),
 ('of', 286598),
 ('to', 264402),
 ('is', 204825),
 ('in', 179158),
 ('i', 132384),
 ('this', 131309),
 ('that', 130096),
 ('it', 129013),
 ('/><br', 100979),
 ('was', 93232),
 ('as', 87679),
 ('with', 84455),
 ('for', 84197),
 ('but', 77832),
 ('on', 62807),
 ('movie', 61431),
 ('are', 56996),
 ('his', 56863),
 ('not', 56568),
 ('you', 55395),
 ('film', 55043),
 ('have', 54383),
 ('he', 51050),
 ('be', 50891),
 ('at', 45133),
 ('one', 44596),
 ('by', 43303),
 ('an', 42098),
 ('they', 40820),
 ('from', 39203),
 ('all', 38462),
 ('who', 38277),
 ('like', 37159),
 ('so', 35840),
 ('just', 34058),
 ('or', 33286),
 ('has', 32596),
 ('about', 32374),
 ('her', 31240),
 ("it's", 30762),
 ('some', 30032),
 ('if', 30015),
 ('out', 28960),
 ('what', 27452),
 ('very', 26820),
 ('when', 26286),
 ('more', 25851),
 ('there', 25602),
 ('would', 23792),
 ('even', 23581),
 ('she', 23415),
 ('good', 23403),
 ('my', 23078),
 ('their', 22603),
 ('only', 

**Stop Words**

For the purpose of sentiment analysis we cannot use the nltk package's predefined stopword list as it has many words like 'not', 'did', etc. which are important for the purpose of analysing the sentiment

Hence I've made a custom list of stop_words which do not have any contribution to the sentiment of a sentence.

In [5]:
stop_words = set(['the','a','and','of','to','in','this','that','on','an','that'])

**Pre-Processing**

For pre-process, we firstly tokenize the sentence and then apply a lemmatizer followed by removal of the stopwords

In [6]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
#from nltk.corpus import stopwords
#eng_stop_words = set(stopwords.words('english'))
wnl = WordNetLemmatizer()
def pre_process(text):
        text = [token for token in word_tokenize(text)]
        text = [wnl.lemmatize(token) for token in text]
        text = [wnl.lemmatize(token,'v') for token in text]
        text = [token for token in text if not token in stop_words]
        text = " ".join(text)
        return text
    


In [7]:
data['review'] = data.review.apply(pre_process)
data.head()

,review,sentiment
0,One other reviewer ha mention after watch just...,positive
1,A wonderful little production . < br / > < br ...,positive
2,I think wa wonderful way spend time too hot su...,positive
3,Basically there 's family where little boy ( J...,negative
4,Petter Mattei 's `` Love Time Money '' be visu...,positive


In [8]:
embed = pd.read_pickle('/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl')


In [9]:
print("Average tokens per example is {0:.0f}".format(np.mean(data['review'].apply(lambda x: len(x.split())))))
print("Max tokens per example is {0:.0f}".format(np.max(data['review'].apply(lambda x: len(x.split())))))
print("Min tokens per example is {0:.0f}".format(np.min(data['review'].apply(lambda x: len(x.split())))))
print("Median tokens per example is {0:.0f}".format(np.median(data['review'].apply(lambda x: len(x.split())))))

Average tokens per example is 232
Max tokens per example is 2393
Min tokens per example is 6
Median tokens per example is 174


In [10]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer(lower = True)


Using TensorFlow backend.


In [11]:
full_text = list((''.join(data['review'].values).split()))
tok.fit_on_texts(full_text)

In [12]:
len(tok.word_index)


100700

In [13]:
train_tok = tok.texts_to_sequences(data['review'])


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(train_tok, maxlen = 230)

In [15]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_enc = encoder.fit_transform(data['sentiment'].values.reshape(-1, 1))
y_enc

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [16]:
dim = 300
NB_WORDS = len(tok.word_index)  #here we consider only the top 30000 most frequently occuring words 
emb_matrix = np.zeros((NB_WORDS+1, dim))
for w, i in tok.word_index.items():
    if i < NB_WORDS:
        vect = embed.get(w)
        if vect is not None:
            emb_matrix[i] = vect

In [17]:
# A simple LSTM 
from keras.layers.wrappers import Bidirectional
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(NB_WORDS+1, dim,weights=[emb_matrix], input_length=230,trainable=False))
model.add(Bidirectional(LSTM(64,dropout=0.2)))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_enc, epochs=10, batch_size=128,validation_split=0.1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 230, 300)          30210300  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 30,397,438
Trainable params: 187,138
Non-trainable params: 30,210,300
_________________________________________________________________
None
Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 146s 3ms/step - loss: 0.4634 - accuracy: 0.7778 - val_loss: 0.3429 - val_accuracy: 0.8526
Epoch 2/10
45000/45000 [==============================] - 144s 3ms/step - loss: 0.3509 - accuracy: 0.8486 - val_loss: 0.2951 - val_accuracy: 0.8792
Epoch 3/10
45000

In [18]:
import pickle
pickle.dump(model,open('LSTM_Model.pkl','wb'))

rev = "I didn't like the movie honestly. The storyline was way too mainstream and the ending was quite expected. One time watch!"
rev = pre_process(rev)
print(rev)
rev_tok = tok.texts_to_sequences(rev)
rev_mod = pad_sequences(rev_tok, maxlen = 130)
model.predict(rev_mod)

rev_tok = tok.texts_to_sequences(["I absolutely hated movie honestly worst The storyline wa way mainstream end wa quite expect One time watch"])
rev_mod = pad_sequences(rev_tok, maxlen = 130)
rev_mod